In [2]:
import sys
import os;
import itertools as it
import numpy as np
import pandas as pd
import pylab as py
from  matplotlib import rc
from matplotlib.colors import LogNorm
%matplotlib inline
from tools.tools import save,load,lprint,load_config
from tools.bar import BAR
from fitlab.resman import RESMAN
from fitlab.parman import PARMAN
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text',usetex=True)
from pandas import read_excel as read
from pandas import DataFrame as df
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import string
# enable plots in the notebook
%matplotlib inline 

In [3]:
conf=load_config('../fitlab/inputs/upol_hermes.py')
conf['resman']=RESMAN(conf)
conf['resman'].get_residuals(conf['parman'].par);

loading sidis data sets 1007

Data vs Theory

In [4]:
UPOL=pd.concat([pd.DataFrame(conf['resman'].sidisres.tabs[idx]) \
              for idx in conf['resman'].sidisres.tabs.keys()])

UPOL.columns

Index([      u'Ebeam',           u'N',          u'Q2',       u'Shift',
             u'alpha',         u'col',      u'hadron',           u'i',
               u'obs',          u'pT', u'r-residuals',   u'residuals',
             u'shift',      u'stat_u',   u'systabs_u',     u'systole',
           u'systrel',      u'target',         u'thy',       u'value',
                 u'x',           u'y',           u'z'],
      dtype='object')

In [5]:

## Grabing list of files
data=os.listdir('../database/sidis/expdata/') # list all files in dir
data=[files for files in data if files.endswith('.xlsx') and not files.startswith('~')] # list of COMPASS data
print "Data files retrieved %s" % data

Data files retrieved ['1000.xlsx', '1001.xlsx', '1002.xlsx', '1003.xlsx', '1004.xlsx', '1005.xlsx', '1006.xlsx', '1007.xlsx', '2000.xlsx', '2001.xlsx', '2002.xlsx', '2003.xlsx', '2004.xlsx', '5001.xlsx', '5002.xlsx']


In [13]:
## Reading and resructuring data

data1000whole = df(read('../database/sidis/expdata/'+data[1]))

# Calculate mesurment error
data1000whole["delta"] = np.sqrt(data1000whole["stat_u"]**2.0+data1000whole["systabs_u"]**2.0)
data1000whole["qT"] = data1000whole["pT"]/data1000whole["z"]
data1000whole["qT2"] = data1000whole["qT"]**2


##Binning data Tick marks for overall 9x9 matrix
xBin=np.array([0.023,0.04,0.055,0.075,0.1,0.14,0.2,0.3,0.4,0.6]) 
Q2Bin=np.array([1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 3.0, 5.0, 15.0])
zBin= np.array([0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 1.1])

 

In [8]:
## Reading and resructuring data for COMPASS

data1000 = UPOL.query('hadron=="pi+"')

# Calculate measurment error
data1000["delta"] = np.sqrt((data1000["stat_u"]**2.0+data1000["systabs_u"]**2.0))
data1000["qT"] = data1000["pT"]/data1000["z"]
data1000["qT2"] = data1000["qT"]**2



##Binning data
xBin=[0.003,0.008,0.013,0.02,0.032,0.055,0.1,0.21,0.4]
Q2Bin=[1,1.7,3,7,16,81]

data1000['xClas'] = pd.cut(data1000['x'], xBin, labels=map(chr, range(97, 97+len(xBin)-1)))
data1000['xBin'] = pd.cut(data1000['x'], xBin)

data1000['Q2Clas'] = pd.cut(data1000['Q2'], Q2Bin, labels=map(chr, range(65, 65+len(Q2Bin)-1)))
data1000['Q2Bin'] = pd.cut(data1000['Q2'], Q2Bin)

print data1000.head()


   Ebeam    N        Q2  Shift     alpha     col hadron    i       obs  \
0   27.6  1.0  1.820673    0.0  0.076435  hermes    pi+  119  M_Hermes   
1   27.6  1.0  1.820673    0.0  0.053237  hermes    pi+  126  M_Hermes   
2   27.6  1.0  1.820673    0.0  0.061018  hermes    pi+  133  M_Hermes   
3   27.6  1.0  1.820673    0.0  0.056770  hermes    pi+  134  M_Hermes   
4   27.6  1.0  1.820673    0.0  0.044043  hermes    pi+  140  M_Hermes   

         pT    ...            x        y         z     delta        qT  \
0  0.101538    ...     0.095738  0.36718  0.223790  0.076435  0.453719   
1  0.101851    ...     0.095738  0.36718  0.274631  0.053237  0.370866   
2  0.100382    ...     0.095738  0.36718  0.334259  0.061018  0.300313   
3  0.203484    ...     0.095738  0.36718  0.334318  0.056770  0.608654   
4  0.100453    ...     0.095738  0.36718  0.419518  0.044043  0.239449   

        qT2  xClas          xBin  Q2Clas     Q2Bin  
0  0.205861      f  (0.055, 0.1]       B  (1.7, 3]  
1  0

In [11]:
data1000['xBin'][0]

0    (0.055, 0.1]
0    (0.055, 0.1]
Name: xBin, dtype: category
Categories (1, object): [(0.055, 0.1]]

In [16]:
# creates index values for final plot matrix
i = np.arange(81).reshape(9,9)
value = data1000whole['value']
qT2 = data1000whole['qT2']
delta = data1000whole['delta']
xClas=range(len(xBin)-1)
Q2Clas=range(len(Q2Bin)-1)
zClas=range(len(zBin)-1)
ind = np.arange(336)

data1000whole['xClas'] = pd.cut(data1000whole['x'], xBin, labels=False)
data1000whole['xBin'] = pd.cut(data1000whole['x'], xBin,labels = False, retbins=0)
xBind = data1000whole['xClas']
xBinned = pd.DataFrame({'xBin': data1000whole['xBin']},index = ind)

data1000whole['Q2Clas'] = pd.cut(data1000whole['Q2'], Q2Bin, labels=False)
data1000whole['Q2Bin'] = pd.cut(data1000whole['Q2'], Q2Bin,labels = False, retbins=0)
Q2Bind = data1000whole['Q2Clas']
Q2Binned = pd.DataFrame({'Q2Bin' : data1000whole['Q2Bin']}, index = ind)

data1000whole['zClas'] = pd.cut(data1000whole['z'], zBin, labels=False)
data1000whole['zBin'] = pd.cut(data1000whole['z'], zBin,labels = False, retbins=0)
zBind = data1000whole['zBin']
zBinned = pd.DataFrame({'zBin' : data1000whole['zBin']}, index = ind)
data_binvalue = data1000whole['value'].as_matrix(columns=None)
data_binqT2 = data1000whole['qT2'].as_matrix(columns=None)
data_bindelta = data1000whole['delta'].as_matrix(columns=None)
data_binz = data1000whole['z'].as_matrix(columns=None)
data_binx = data1000whole['x'].as_matrix(columns=None)

# Creates the index needed to create f and g DataFrames
ind = np.arange(336)
Gridmap = np.arange(80)

X = data1000whole['pT']

h = pd.DataFrame({'xBin': xBind, 'Q2Bin': Q2Bind, 'zBin' : zBind},index=ind)
# Creates a DataFrame with qT2 as x, value as y
g = pd.DataFrame({'x': data_binx, 'value': data_binvalue, 'z' : data_binz, 'delta' : data_bindelta, 'qT2' :
    data_binqT2},index=ind)
xbin0 = xBinned.query('xBin == 0')
x0ind  = xbin0.index.values
new_xbin0 = X.iloc[x0ind]
xbin1 = xBinned.query('xBin == 1')
x1ind  = xbin1.index.values
new_xbin1 = X.iloc[x1ind]
xbin2 = xBinned.query('xBin == 2')
x2ind  = xbin2.index.values
new_xbin2 = X.iloc[x2ind]
xbin3 = xBinned.query('xBin == 3')
x3ind  = xbin3.index.values
new_xbin3 = X.iloc[x3ind]
xbin4 = xBinned.query('xBin == 4')
x4ind  = xbin4.index.values
new_xbin4 = X.iloc[x4ind]
xbin5 = xBinned.query('xBin == 5')
x5ind  = xbin5.index.values
new_xbin5 = X.iloc[x5ind]
xbin6 = xBinned.query('xBin == 6')
x6ind  = xbin6.index.values
new_xbin6 = X.iloc[x6ind]
xbin7 = xBinned.query('xBin == 7')
x7ind  = xbin7.index.values
new_xbin7 = X.iloc[x7ind]
xbin8 = xBinned.query('xBin == 8')
x8ind  = xbin8.index.values
new_xbin8 = X.iloc[x8ind]

# x axis label is pT for xbin
# y axis label is value for Q2 bin

xind = {'0':x0ind,'1':x1ind,'2':x2ind,'3':x3ind,'4':x4ind,'5':x5ind,'6':x6ind,'7':x7ind,'8':x8ind}

#Here is binned data for pT back in a DataFrame 
pTdat = pd.DataFrame({'0':new_xbin0,'1':new_xbin1,'2':new_xbin2,'3':new_xbin3,
                        '4':new_xbin4,'5':new_xbin5,'6':new_xbin6,'7':new_xbin7,
                        '8':new_xbin8},index = ind)
    
Y = data1000whole['value']    
Q2bin0 = Q2Binned.query('Q2Bin == 0')
q0ind  = Q2bin0.index.values
value0 = Y.iloc[q0ind]
Q2bin1 = Q2Binned.query('Q2Bin == 1')
q1ind  = Q2bin1.index.values
value1 = Y.iloc[q1ind]
Q2bin2 = Q2Binned.query('Q2Bin == 2')
q2ind  = Q2bin2.index.values
value2 = Y.iloc[q2ind]
Q2bin3 = Q2Binned.query('Q2Bin == 3')
q3ind  = Q2bin3.index.values
value3 = Y.iloc[q3ind]
Q2bin4 = Q2Binned.query('Q2Bin == 4')
q4ind  = Q2bin4.index.values
value4 = Y.iloc[q4ind]
Q2bin5 = Q2Binned.query('Q2Bin == 5')
q5ind  = Q2bin5.index.values
value5 = Y.iloc[q5ind]
Q2bin6 = Q2Binned.query('Q2Bin == 6')
q6ind  = Q2bin6.index.values
value6 = Y.iloc[q6ind]
Q2bin7 = Q2Binned.query('Q2Bin == 7')
q7ind  = Q2bin7.index.values
value7 = Y.iloc[q7ind]
Q2bin8 = Q2Binned.query('Q2Bin == 8')
q8ind  = Q2bin8.index.values
value8 = Y.iloc[q8ind]
#Here is binned data for value back in a DataFrame 
valuedat = pd.DataFrame({'0':value0,'1':value1,'2':value2,'3':value3,
                        '4':value4,'5':value5,'6':value6,'7':value7,
                        '8':value8},index=ind)

# I didnt use vdat in the end I used valuedat
vdat = {'0':value0,'1':value1,'2':value2,'3':value3,
                        '4':value4,'5':value5,'6':value6,'7':value7,
                        '8':value8}


Z = data1000whole['z']
zbin0 = zBinned.query('zBin == 0')
z0ind  = zbin0.index.values

zbin1 = zBinned.query('zBin == 1')
z1ind  = zbin1.index.values

zbin2 = zBinned.query('zBin == 2')
z2ind  = zbin2.index.values

zbin3 = zBinned.query('zBin == 3')
z3ind  = zbin3.index.values

zbin4 = zBinned.query('zBin == 4')
z4ind  = zbin4.index.values

zbin5 = zBinned.query('zBin == 5')
z5ind  = zbin5.index.values


z = {'0':z0ind,'1':z1ind,'2':z2ind,'3':z3ind,'4':z4ind,'5':z5ind}

k = 0
fig=py.figure(figsize=(15, 15),facecolor="white")
for f in pTdat:       
    for j in valuedat:
        if j == '8':
            k = int(f) +1
        elif j == '7':
            k = 10 + int(f)
        elif j == '6':
            k = 19 + int(f)
        elif j == '5':
            k = 28 + int(f)
        elif j == '4':
            k = 37 + int(f)
        elif j == '3':
            k = 46 + int(f)
        elif j == '2':
            k = 55 + int(f)
        elif j == '1':
            k = 64 + int(f)
        elif j == '0':
            k = 73 + int(f)
        ax = py.subplot(9,9,k)
        for i in z:
            zindex = z[i]
            xdat  = pTdat[f].iloc[zindex]
            xdat = xdat.dropna()
            ydat = valuedat[j].iloc[zindex]
            ydat = ydat.dropna()
           
            ddat = delta.iloc[zindex]
            ddat = ddat.dropna()                                                    
            databin = pd.DataFrame({'x':xdat,'y':ydat,'d':ddat})
            databin = databin.dropna()
            
            if databin.empty:
                pass
            else:
                print('xbin = ' + str(f))
                print('ybin = ' + str(j))
                print('bin'+ str(i), databin)
            #if ydat.empty or xdat.empty:
            #    pass
            # else:
            #if len(databin['y']) != len(databin['x']):
            #    pass
            #else:
            ax.errorbar(databin['x'],databin['y'],yerr=databin['d'],capsize=6)
            ax.set_yscale("log")
                   
            ax.grid()
                    
ax.set_ylabel(r"$Q^2$ bins",rotation="horizontal")
ax.set_xlabel(r"$x$ bins")

ValueError: Shape of passed values is (5, 335), indices imply (5, 336)